In [1]:

import requests
import pandas as pd
import hopsworks
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util

warnings.filterwarnings("ignore")

In [2]:
load_dotenv()
proj = hopsworks.login()
fs = proj.get_feature_store() 

2025-01-02 16:14:24,322 INFO: Initializing external client
2025-01-02 16:14:24,322 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-02 16:14:25,635 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164449


In [3]:
el_prices_fg = fs.get_feature_group(
    name='el_prices',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
power_fg = fs.get_feature_group(
    name='power',
    version=1,
)

In [7]:
date = datetime.date.today()
yesterday = date - timedelta(days=1)
el_price_df = util.get_el_price(yesterday)
el_price_df['date'] = pd.to_datetime(el_price_df["date"])
el_price_df

,price,date
0,0.075346,2025-01-01


In [7]:
el_prices_fg.insert(el_price_df, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: el_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164449/jobs/named/el_prices_1_offline_fg_materialization/executions
2025-01-02 16:06:33,035 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-02 16:06:45,681 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-02 16:08:36,239 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-02 16:08:36,380 INFO: Waiting for log aggregation to finish.
2025-01-02 16:08:54,802 INFO: Execution finished successfully.


(Job('el_prices_1_offline_fg_materialization', 'SPARK'), None)

In [4]:
latitude = 59.329323
longitude = 18.068581

weather_data = util.get_daily_weather(1, 0, latitude, longitude)

weather_data.head()

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                       date  temperature_2m_mean  sunshine_duration  \
0 2025-01-01 00:00:00+00:00             0.015417                0.0   

   precipitation_sum  wind_speed_10m_max  wind_direction_10m_dominant  
0               10.5           18.719999                     3.529286  


,date,temperature_2m_mean,sunshine_duration,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
0,2025-01-01 00:00:00+00:00,0.015417,0.0,10.5,18.719999,3.529286


In [5]:
weather_fg.insert(weather_data, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164449/jobs/named/weather_1_offline_fg_materialization/executions
2025-01-02 16:14:53,454 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-02 16:14:56,629 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-02 16:16:22,027 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-02 16:16:22,172 INFO: Waiting for log aggregation to finish.
2025-01-02 16:16:30,706 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'), None)

In [12]:
load_dotenv()
ELECTRICITY_API_TOKEN = os.getenv("ELECTRICTY_MAP_API_KEY")
power_data = util.fetch_data_for_yesterday(ELECTRICITY_API_TOKEN)
yesterday_formated = yesterday.strftime("%d.%m.%Y")
power_data.insert(0, "date", yesterday_formated)
power_data['date'] = pd.to_datetime(el_price_df["date"])
print(power_data)

Data fetched successfully:
   hydro_mw  nuclear_mw    other_mw      wind_mw
0  8191.875     5821.75  515.916667  6341.791667
        date  hydro_mw  nuclear_mw    other_mw      wind_mw
0 2025-01-01  8191.875     5821.75  515.916667  6341.791667


In [13]:
power_fg.insert(power_data, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: power_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164449/jobs/named/power_1_offline_fg_materialization/executions
2025-01-02 16:26:52,922 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-02 16:26:56,089 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-02 16:28:37,175 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-02 16:28:37,318 INFO: Waiting for log aggregation to finish.
2025-01-02 16:28:45,823 INFO: Execution finished successfully.


(Job('power_1_offline_fg_materialization', 'SPARK'), None)

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
                       date  temperature_2m_max  sunshine_duration  \
0 2024-12-24 23:00:00+00:00            9.217500           0.000000   
1 2024-12-25 23:00:00+00:00            9.117499           0.000000   
2 2024-12-26 23:00:00+00:00            7.967500        1492.621582   
3 2024-12-27 23:00:00+00:00            6.717500           0.000000   
4 2024-12-28 23:00:00+00:00            7.917500           0.000000   
5 2024-12-29 23:00:00+00:00            7.667500       13238.261719   
6 2024-12-30 23:00:00+00:00            0.917500       13910.627930   

   precipitation_sum  wind_speed_10m_max  wind_direction_10m_dominant  
0                0.2           20.160000                   240.726639  
1                0.0           24.119999                   264.867737  
2                3.4           16.919998                   277.060303  
3       